# Change Point Detection

This notebook is designed to perform bayesian change point detection on each individual token.
This method provides probability estimates that a changepoint occurred at a given time point.
There may be a problem with detecting changes at the extreme ends of the time series but inconclusive at the moment.

In [1]:
from functools import partial
from pathlib import Path
import pickle
import re

from gensim.models import Word2Vec
import numpy as np
import pandas as pd
import plotnine as p9
import plydata as ply
import plydata.tidy as ply_tdy
import tqdm

import bayesian_changepoint_detection.offline_changepoint_detection as offcd
from detecta import detect_cusum

Use scipy logsumexp().


# Grab Word Frequencies

In [2]:
if not Path("output/all_tok_frequencies.tsv.xz").exists():
    data = []
    for model in tqdm.tqdm(
        sorted(
            list(Path("../multi_model_experiment/output/models").rglob("*/*_0.model"))
        )
    ):
        wv_model = Word2Vec.load(str(model))
        word_freq_df = pd.DataFrame.from_records(
            [
                {
                    "tok": re.escape(tok),
                    "word_count": wv_model.wv.get_vecattr(tok, "count"),
                    "year": re.search(r"(\d+)_0", str(model)).group(1),
                }
                for tok in wv_model.wv.index_to_key
            ]
        )

        total_count = (word_freq_df >> ply.pull("word_count")).sum()

        word_freq_df = word_freq_df >> ply.define(frequency=f"word_count/{total_count}")

        data.append(word_freq_df)

In [3]:
if not Path("output/all_tok_frequencies.tsv.xz").exists():
    all_word_freq_df = pd.concat(data) >> ply.call(".dropna")
    (
        all_word_freq_df
        >> ply.call(".dropna")
        >> ply.query("year != ''")
        >> ply.call(".astype", {"year": int, "frequency": float, "word_count": int})
        >> ply.call(
            ".to_csv",
            "output/all_tok_frequencies.tsv.xz",
            sep="\t",
            index=False,
            compression="xz",
        )
    )

else:
    all_word_freq_df = (
        pd.read_csv("output/all_tok_frequencies.tsv.xz", sep="\t", na_filter=False)
        >> ply.call(".dropna")
        >> ply.query("year != ''")
        >> ply.call(".astype", {"year": int, "frequency": float, "word_count": int})
    )

print((all_word_freq_df >> ply.select("tok") >> ply.distinct()).shape)
all_word_freq_df

/home/danich1/anaconda3/envs/biovectors/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3156: DtypeWarning: Columns (1,2,3) have mixed types.Specify dtype option on import or set low_memory=False.


(3744412, 1)


,tok,word_count,year,frequency
0,\,87283,2000,8.850373e-02
1,the,38877,2000,3.942073e-02
2,of,35677,2000,3.617598e-02
3,",",33131,2000,3.359437e-02
4,\.,32263,2000,3.271423e-02
...,...,...,...,...
8556728,1\-ethynyladamantanes,1,2020,1.285818e-08
8556729,1800\-fold,1,2020,1.285818e-08
8556730,foveally,1,2020,1.285818e-08
8556731,0\.45x:12\.79,1,2020,1.285818e-08


## Filter out Tokens

In [4]:
# plydata implicitly groups datapoints
# need to revert to pandas to actually cycle through each group
token_group = all_word_freq_df >> ply.call(".groupby", "tok")

In [5]:
if not Path("output/token_frequency_dict.pkl").exists():
    all_token_map = dict()
    for tok, group in tqdm.tqdm(token_group):
        all_token_map[tok] = group >> ply.arrange("year") >> ply.pull("year")

    pickle.dump(all_token_map, open("output/token_frequency_dict.pkl", "wb"))
else:
    all_token_map = pickle.load(open("output/token_frequency_dict.pkl", "rb"))

In [6]:
cleared_token_map = {
    tok: all_token_map[tok]
    for tok in tqdm.tqdm(all_token_map)
    if len(all_token_map[tok]) > 1
    and np.diff(all_token_map[tok]).sum() == len(all_token_map[tok]) - 1
}
print(len(cleared_token_map))

100%|██████████| 3744412/3744412 [00:09<00:00, 402887.58it/s]

201962


## Calculate frequency ratio

In [7]:
all_word_frequency_ratio_df = (
    all_word_freq_df
    >> ply.query(f"tok in {list(cleared_token_map.keys())}")
    >> ply.group_by("tok")
    >> ply.arrange("year")
    >> ply.define(
        frequency_ratio=lambda x: x.frequency / x.shift(1).frequency,
    )
    >> ply.ungroup()
    >> ply.call(".dropna")
    >> ply.define(year=lambda x: x["year"].apply(lambda y: f"{y-1}-{y}"))
)
print((all_word_frequency_ratio_df >> ply.select("tok") >> ply.distinct()).shape)
all_word_frequency_ratio_df.head()

(201962, 1)


,tok,word_count,year,frequency,frequency_ratio
55842,traditionally,4,2000-2001,0.000003,0.557732
55837,omental,4,2000-2001,0.000003,0.478056
55834,418,4,2000-2001,0.000003,0.836598
55833,175,4,2000-2001,0.000003,0.304218
55830,\-b,4,2000-2001,0.000003,0.836598


# Grab semantic change values

In [8]:
distance_files = list(
    Path("../multi_model_experiment/output/combined_inter_intra_distances").rglob(
        "saved_*_distance.tsv"
    )
)
print(len(distance_files))

20


In [9]:
year_distance_map = {
    re.search(r"\d+", str(year_file)).group(0): (
        pd.read_csv(str(year_file), sep="\t", na_filter=False)
    )
    for year_file in tqdm.tqdm(distance_files)
}

100%|██████████| 20/20 [00:01<00:00, 10.93it/s]


In [10]:
full_token_set_df = (
    pd.concat([year_distance_map[year] for year in tqdm.tqdm(year_distance_map)])
    >> ply.define(tok=lambda x: x.tok.apply(lambda y: re.escape(y)))
    >> ply_tdy.unite("year", "year_1", "year_2", sep="-")
)
print((full_token_set_df >> ply.select("tok") >> ply.distinct()).shape)
full_token_set_df.head()

100%|██████████| 20/20 [00:00<00:00, 48072.25it/s]


(327733, 1)


,tok,original_global_distance,global_distance_qst,ratio_metric,year
0,poly\(adp,0.208324,0.499768,0.999074,2013-2014
1,partners,0.140676,0.467454,0.877771,2013-2014
2,permeation,0.188546,0.513674,1.056235,2013-2014
3,137,0.283853,0.557411,1.259432,2013-2014
4,sixty,0.137166,0.468312,0.880801,2013-2014


In [11]:
freq_map = dict()
for tok, group in tqdm.tqdm(full_token_set_df.groupby("tok")):
    freq_map[tok] = group >> ply.arrange("year") >> ply.pull("ratio_metric")

100%|██████████| 327733/327733 [06:55<00:00, 788.04it/s]


# Combine both information

Using an idea similar to SCAF \[[1](https://doi.org/10.1007/s00799-019-00271-6)\], I'm combining the global qst metric with the percent change in frequency.
SCAF uses percent change for both metrics; however, the caveat is that their method loses information for the first two timepoints.
In my case given that global_distance_qst is a metric that's bound between 0 and 1 frequency percent change can be used directly.
By combining these two terms I can use this metric as a means to estimate change and allow for bayesian changepoint detection to calculate the probability of a timepoint change.

In [12]:
data_rows = list()
for tok, group in tqdm.tqdm(all_word_frequency_ratio_df.groupby("tok")):
    if tok in freq_map and tok != "null":
        data_rows.append(
            {
                "ratio_metric": (freq_map[tok]),
                "frequency_ratio": (
                    group >> ply.arrange("year") >> ply.pull("frequency_ratio")
                ),
                "year": (group >> ply.arrange("year") >> ply.pull("year")),
                "tok": [tok]
                * (group >> ply.arrange("year") >> ply.pull("year")).shape[0],
            }
        )
print(len(data_rows))

100%|██████████| 201962/201962 [05:00<00:00, 672.27it/s] 


80513


In [13]:
final_token_df = pd.concat([pd.DataFrame.from_dict(data) for data in data_rows])
final_token_df

,ratio_metric,frequency_ratio,year,tok
0,2.250108,1.003918,2000-2001,!
1,1.685207,0.864842,2001-2002,!
2,2.333010,0.646562,2002-2003,!
3,1.749069,2.505442,2003-2004,!
4,3.092308,1.427221,2004-2005,!
...,...,...,...,...
16,1.347119,3.111310,2016-2017,zyxin
17,1.345246,0.266405,2017-2018,zyxin
18,1.304166,1.725867,2018-2019,zyxin
19,1.353653,0.745896,2019-2020,zyxin


In [14]:
merged_frequency_df = (
    final_token_df
    >> ply.rename(year_pair="year")
    >> ply.select(
        "tok",
        "ratio_metric",
        "year_pair",
        "frequency_ratio",
    )
)
print((merged_frequency_df >> ply.select("tok") >> ply.distinct()).shape)
merged_frequency_df

(80513, 1)


,tok,ratio_metric,year_pair,frequency_ratio
0,!,2.250108,2000-2001,1.003918
1,!,1.685207,2001-2002,0.864842
2,!,2.333010,2002-2003,0.646562
3,!,1.749069,2003-2004,2.505442
4,!,3.092308,2004-2005,1.427221
...,...,...,...,...
16,zyxin,1.347119,2016-2017,3.111310
17,zyxin,1.345246,2017-2018,0.266405
18,zyxin,1.304166,2018-2019,1.725867
19,zyxin,1.353653,2019-2020,0.745896


In [15]:
change_metric_df = (
    merged_frequency_df
    >> ply.group_by("tok")
    >> ply.arrange("year_pair")
    >> ply.define(
        change_metric_ratio="ratio_metric + frequency_ratio",
    )
    >> ply.ungroup()
    >> ply.select(
        "year_pair", "tok", "ratio_metric", "frequency_ratio", "change_metric_ratio"
    )
)
change_metric_df >> ply.call(
    ".to_csv", "output/change_metric_abstracts.tsv", sep="\t", index=False
)
change_metric_df

,year_pair,tok,ratio_metric,frequency_ratio,change_metric_ratio
0,2000-2001,!,2.250108,1.003918,3.254026
0,2000-2001,drop,2.380378,0.988707,3.369085
0,2000-2001,cfu,1.886854,0.717084,2.603938
0,2000-2001,interaction,1.765145,0.851699,2.616845
0,2000-2001,m2,2.155668,1.055401,3.211069
...,...,...,...,...,...
0,2019-2020,th17\)/regulatory,1.186573,0.779800,1.966373
13,2019-2020,th17,1.052309,0.947320,1.999629
19,2019-2020,ruffles,1.356828,0.371333,1.728161
15,2019-2020,ligamentum,1.139380,0.995490,2.134869


# Change point detection

## Bayesian

Use Semantic Change Analysis with Frequency (SCAF) to perform bayesian change point detection.

In [16]:
if not Path("output/bayesian_changepoint_data_abstracts.tsv").exists():
    change_point_results = []
    for tok, tok_series_df in tqdm.tqdm(change_metric_df.groupby("tok")):

        change_metric_ratio = tok_series_df >> ply.pull("change_metric_ratio")
        Q, P, Pcp = offcd.offline_changepoint_detection(
            change_metric_ratio,
            partial(offcd.const_prior, l=(len(change_metric_ratio) + 1)),
            offcd.gaussian_obs_log_likelihood,
            truncate=-40,
        )
        estimated_changepoint_probability = np.exp(Pcp).sum(0)

        change_point_results.append(
            pd.DataFrame.from_dict(
                {
                    "tok": [tok] * len(estimated_changepoint_probability),
                    "changepoint_prob": estimated_changepoint_probability,
                    "year_pair": (
                        tok_series_df
                        >> ply.select("year_pair")
                        >> ply.call(".shift", -1)
                        >> ply.call(".dropna")
                        >> ply.pull("year_pair")
                    ),
                }
            )
        )

In [17]:
if not Path("output/bayesian_changepoint_data_abstracts.tsv", sep="\t").exists():
    change_point_df = pd.concat(change_point_results)
    change_point_df.to_csv(
        "output/bayesian_changepoint_data_abstracts.tsv", sep="\t", index=False
    )
else:
    change_point_df = pd.read_csv(
        "output/bayesian_changepoint_data_abstracts.tsv", sep="\t"
    )
change_point_df.head()

,tok,changepoint_prob,year_pair
0,!,0.004488,2001-2002
1,!,0.004065,2002-2003
2,!,0.002007,2003-2004
3,!,0.003899,2004-2005
4,!,0.182222,2005-2006


In [18]:
(change_point_df >> ply.arrange("-changepoint_prob") >> ply.slice_rows(30))

,tok,changepoint_prob,year_pair
2200,014,0.995481,2014-2015
639411,sars,0.994003,2018-2019
2198,014,0.993782,2012-2013
2226,016,0.993016,2016-2017
348190,h7n9,0.988336,2013-2014
224053,coronavirus,0.981903,2018-2019
2267,020,0.980726,2018-2019
265875,doi:10\.1186,0.978057,2014-2015
289640,enzalutamide,0.976665,2013-2014
262549,distancing,0.972938,2018-2019


## CUSUM

This section uses the CUSUM algorithm to determine changepoint events from time series. This algorithm uses a threshold value to determine cutoffs for a changepoint event. To figure out this value I'm using a cutoff of 2 standard deviations from the mean. The mean is calculated after filtering out one outlier that contains a value of 5000 when the other values are 700 and less. This allows for more potential matches to be found.

In [19]:
cutoff_df = (
    change_metric_df
    >> ply.query("change_metric_ratio < 1000")
    >> ply.select("change_metric_ratio")
    >> ply.call(".describe")
)
cutoff_df

,change_metric_ratio
count,843653.000000
mean,2.712828
std,1.935849
min,0.623139
25%,1.991638
50%,2.316084
75%,2.898272
max,726.176448


In [20]:
threshold = cutoff_df.loc["mean"] * cutoff_df.loc["std"] * 2
threshold[0]

10.503251064958798

In [21]:
if not Path("output/cusum_changepoint_abstracts.tsv").exists():
    change_point_results = []
    for tok, tok_series_df in tqdm.tqdm(change_metric_df.groupby("tok")):

        change_metric_ratio = tok_series_df >> ply.pull("change_metric_ratio")
        year_series = tok_series_df >> ply.pull("year_pair")

        alarm, start, end, amp = detect_cusum(
            change_metric_ratio,
            threshold=threshold[0],
            drift=0.5,
            ending=True,
            show=False,
        )

        for values in zip(alarm, start, end, amp):

            change_point_results.append(
                {
                    "tok": tok,
                    "changepoint_idx": year_series[values[0]],
                    "start_idx": year_series[values[1]],
                    "end_idx": year_series[values[2]],
                    "value": values[3],
                }
            )

100%|██████████| 80513/80513 [00:28<00:00, 2808.16it/s]


In [22]:
if not Path("output/cusum_changepoint_abstracts.tsv").exists():
    change_point_df = pd.DataFrame.from_records(change_point_results)
    change_point_df >> ply.call(
        ".to_csv", "output/cusum_changepoint_abstracts.tsv", sep="\t", index=False
    )
else:
    change_point_df = pd.read_csv("output/cusum_changepoint_abstracts.tsv", sep="\t")
print((change_point_df >> ply.select("tok") >> ply.distinct()).shape)
change_point_df

(2682, 1)


,tok,changepoint_idx,start_idx,end_idx,value
0,/5\-,2016-2017,2015-2016,2016-2017,11.948682
1,/5\-,2017-2018,2016-2017,2017-2018,-11.968209
2,/disease,2017-2018,2016-2017,2017-2018,12.327052
3,/disease,2018-2019,2017-2018,2018-2019,-12.264762
4,/ftc,2011-2012,2010-2011,2011-2012,11.435107
...,...,...,...,...,...
4623,zrc,2018-2019,2017-2018,2018-2019,25.106426
4624,zrc,2019-2020,2018-2019,2019-2020,-24.387509
4625,zu,2019-2020,2018-2019,2019-2020,27.577021
4626,zumba,2017-2018,2016-2017,2017-2018,16.246502


In [23]:
(change_point_df >> ply.arrange("-value") >> ply.slice_rows(30))

,tok,changepoint_idx,start_idx,end_idx,value
10,014,2013-2014,2011-2012,2013-2014,393.790997
22,10\.1186,2016-2017,2013-2014,2016-2017,347.708598
12,016,2015-2016,2013-2014,2015-2016,336.462453
1847,h7n9,2012-2013,2011-2012,2012-2013,249.456837
3777,sars,2019-2020,2018-2019,2019-2020,145.519452
14,020,2019-2020,2017-2018,2019-2020,134.201314
777,cas9,2012-2013,2011-2012,2012-2013,110.888604
48,2003,2002-2003,2001-2002,2002-2003,99.589218
50,2004,2003-2004,2001-2002,2003-2004,80.950142
674,breastfeeding,2002-2003,2001-2002,2002-2003,76.576341


# Take Home Points

1. Bayesian change point detection provides insight on the specific year period a semantic change point may have occurred.
2. Best positive result is pandemic which underwent a focus shift from bird flu and influenza to coronavirus.
3. Follow up analysis which will appear in the next notebook will involve looking at the top X token neighbors to the query word. By doing that one can estimate which kind of shift a word has undergone.